In [1]:
from fyers_apiv3 import fyersModel
import pandas as pd
from fyers_apiv3.FyersWebsocket import data_ws
import datetime as dt
import time 
import numpy as np
import math
import json
import pyotp
from urllib import parse
import sys
from datetime import datetime, timedelta
import pandas_ta as ta
import matplotlib.pyplot as plt
import requests



In [ ]:
import pandas as pd 
import numpy as np
import datetime
import Analysis
import pandas_ta as ta
from nselib import capital_market
from nselib import derivatives
import yfinance as yf

In [2]:
FY_ID = "YJ08526"
APP_ID_TYPE = "2"
TOTP_KEY = "W3UOSVQQO2V4KJT3RRXYJDHFBKP3372N"
PIN = "7776"
APP_ID = "ETYHMVN1Q6"
REDIRECT_URI = "https://www.google.com/"
APP_TYPE = "100"
APP_ID_HASH = 'dc1cd1f989e67ad53623c26ac48315e1e88ed60bdd58a2cbc12ea294c248d3db'
response_type = "code"
grant_type = "authorization_code"

BASE_URL = "https://api-t2.fyers.in/vagator/v2"
BASE_URL_2 = "https://api-t1.fyers.in/api/v3"
URL_SEND_LOGIN_OTP = BASE_URL + "/send_login_otp"
URL_VERIFY_TOTP = BASE_URL + "/verify_otp"
URL_VERIFY_PIN = BASE_URL + "/verify_pin"
URL_TOKEN = BASE_URL_2 + "/token"
URL_VALIDATE_AUTH_CODE = BASE_URL_2 + "/validate-authcode"

SUCCESS = 1
ERROR = -1

In [3]:
def verify_client_id(CLIENT_ID,app_id):
    try:
        payload = {
            "fy_id": CLIENT_ID,
            "app_id": app_id
        }

        result_string = requests.post(url=URL_SEND_LOGIN_OTP, json=payload)
        # print("result_string : ", result_string.text)
        if result_string.status_code != 200:
            return [ERROR, result_string.text]

        result = json.loads(result_string.text)
        request_key = result["request_key"]

        return [SUCCESS, request_key]
    
    except Exception as e:
        return [ERROR, e]

In [4]:
def generate_totp(TOTP_KEY):
    try:
        generated_totp = pyotp.TOTP(TOTP_KEY).now()
        return [SUCCESS, generated_totp]
    
    except Exception as e:
        return [ERROR, e]

In [14]:
def verify_totp(request_key, totp):
    try:
        payload = {
            "request_key": request_key,
            "otp": totp
        }

        result_string = requests.post(url=URL_VERIFY_TOTP, json=payload)
        if result_string.status_code != 200:
            return [ERROR, result_string.text]

        result = json.loads(result_string.text)
        request_key = result["request_key"]

        return [SUCCESS, request_key]
    
    except Exception as e:
        return [ERROR, e]

In [5]:
def verify_PIN(request_key, pin):
    try:
        payload = {
            "request_key": request_key,
            "identity_type": "pin",
            "identifier": pin
        }

        result_string = requests.post(url=URL_VERIFY_PIN, json=payload)
        if result_string.status_code != 200:
            return [ERROR, result_string.text]
    
        result = json.loads(result_string.text)
        access_token = result["data"]["access_token"]

        return [SUCCESS, access_token]
    
    except Exception as e:
        return [ERROR, e]

In [6]:

def token(fy_id, app_id, redirect_uri, app_type, access_token):
    try:
        payload = {
            "fyers_id": fy_id,
            "app_id": app_id,
            "redirect_uri": redirect_uri,
            "appType": app_type,
            "code_challenge": "",
            "state": "sample_state",
            "scope": "",
            "nonce": "",
            "response_type": "code",
            "create_cookie": True
        }
        headers={'Authorization': f'Bearer {access_token}'}

        result_string = requests.post(
            url=URL_TOKEN, json=payload, headers=headers
        )

        if result_string.status_code != 308:
            return [ERROR, result_string.text]

        result = json.loads(result_string.text)
        url = result["Url"]
        auth_code = parse.parse_qs(parse.urlparse(url).query)['auth_code'][0]

        return [SUCCESS, auth_code]
    
    except Exception as e:
        return [ERROR, e]

In [7]:
def validate_authcode(auth_code):
    try:
        payload = {
            "grant_type": "authorization_code",
            "appIdHash": APP_ID_HASH,
            "code": auth_code,
        }

        result_string = requests.post(url=URL_VALIDATE_AUTH_CODE, json=payload)
        if result_string.status_code != 200:
            return [ERROR, result_string.text]

        result = json.loads(result_string.text)
        access_token = result["access_token"]

        return [SUCCESS, access_token]
    
    except Exception as e:
        return [ERROR, e]

In [21]:
def main():
    # Step 1 - Retrieve request_key from verify_client_id Function
    verify_client_id_result = verify_client_id(FY_ID,APP_ID)
    if verify_client_id_result[0] != SUCCESS:
        print(f"verify_client_id failure - {verify_client_id_result[1]}")
        sys.exit()
    else:
        print("verify_client_id success")

    # Step 2 - Generate totp
    generate_totp_result = generate_totp(TOTP_KEY)
    if generate_totp_result[0] != SUCCESS:
        print(f"generate_totp failure - {generate_totp_result[1]}")
        sys.exit()
    else:
        print("generate_totp success")

    # Step 3 - Verify totp and get request key from verify_totp Function.
    request_key = verify_client_id_result[1]
    totp = generate_totp_result[1]
    verify_totp_result = verify_totp(request_key=request_key, totp=totp)
    if verify_totp_result[0] != SUCCESS:
        print(f"verify_totp_result failure - {verify_totp_result}")
        sys.exit()
    else:
        print("verify_totp_result success")

    # Step 4 - Verify pin and send back access token
    request_key_2 = verify_totp_result[1]
    verify_pin_result = verify_PIN(request_key=request_key_2, pin=PIN)
    if verify_pin_result[0] != SUCCESS:
        print(f"verify_pin_result failure - {verify_pin_result[1]}")
        sys.exit()
    else:
        print("verify_pin_result success")

    # Step 5 - Get auth code for API V3 App from trade access token
    token_result = token(
        fy_id=FY_ID, app_id=APP_ID, redirect_uri=REDIRECT_URI, app_type=APP_TYPE,
        access_token=verify_pin_result[1]
    )
    if token_result[0] != SUCCESS:
        print(f"token_result failure - {token_result[1]}")
        sys.exit()
    else:
        print("token_result success")

    # Step 6 - Get API V3 access token from validating auth code
    auth_code = token_result[1]
    validate_authcode_result = validate_authcode(auth_code=auth_code)
    if token_result[0] != SUCCESS:
        print(f"validate_authcode failure - {validate_authcode_result[1]}")
        sys.exit()
    else:
        print("validate_authcode success")
    
    access_token = APP_ID + "-" + APP_TYPE + ":" + validate_authcode_result[1]

    print(f"\naccess_token - {access_token}\n")

    return access_token
    

In [22]:
main()

verify_client_id success
generate_totp success
verify_totp_result success
verify_pin_result success
token_result success
validate_authcode success

access_token - ETYHMVN1Q6-100:eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb0JLMS1OQ0lXUGFzcldjLU8zelBGamQwTTlMWXJtNlljalB4eHM2eVp2NFZBLTZMVDl1QVlHZmhJdFpwQmFVS243SUxweG43RXFXZjdRUUNIYTBUS3NPb2Zack9OWWVSemg1R01vM0tTa1NvSVFvMD0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJkOWE5ZDc2MWI2MzM0M2IzMGQ0NGVhNzQ5ZjAyNWFmMTZiZWNlODZlMjc5Mjg3NTc3ZmRlYTVlMCIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWUowODUyNiIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzQ1MTk1NDAwLCJpYXQiOjE3NDUxMzcwMjIsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc0NTEzNzAyMiwic3ViIjoiYWNjZXNzX3Rva2VuIn0.IdqaWWGayx00hGrq3vH5yNnZWJ2Pvs4ikO2Pf6qKmMU



'ETYHMVN1Q6-100:eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb0JLMS1OQ0lXUGFzcldjLU8zelBGamQwTTlMWXJtNlljalB4eHM2eVp2NFZBLTZMVDl1QVlHZmhJdFpwQmFVS243SUxweG43RXFXZjdRUUNIYTBUS3NPb2Zack9OWWVSemg1R01vM0tTa1NvSVFvMD0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJkOWE5ZDc2MWI2MzM0M2IzMGQ0NGVhNzQ5ZjAyNWFmMTZiZWNlODZlMjc5Mjg3NTc3ZmRlYTVlMCIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWUowODUyNiIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzQ1MTk1NDAwLCJpYXQiOjE3NDUxMzcwMjIsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc0NTEzNzAyMiwic3ViIjoiYWNjZXNzX3Rva2VuIn0.IdqaWWGayx00hGrq3vH5yNnZWJ2Pvs4ikO2Pf6qKmMU'

In [23]:
access_token = main()
data = {"access_token": access_token}
with open('access_token.json', 'w') as json_file:
    json.dump(data, json_file)

verify_client_id success
generate_totp success
verify_totp_result success
verify_pin_result success
token_result success
validate_authcode success

access_token - ETYHMVN1Q6-100:eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCb0JLMkJVSFJ0dUh2MW5BTzkzY0hxdWpmWmpCejFmeFdlV3RmcUtzUTZWX19peXZDdmpibFBabWVUdkc0UUp2MjRWdlJaNlBNNTkyV1lYMkViRXduSXE5TVU5eXdob0g1cFI3Ml9FeGk0bV96RTRFST0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiI3NTU2N2IzODdhYzg5YTJmNzExMjY1ZjMyMjljMzhiZTFlYWE2ODBhZDVkZDA3ZGU0M2Y4ZDQ5MCIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWUowODUyNiIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzQ1MTk1NDAwLCJpYXQiOjE3NDUxMzcwMjUsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc0NTEzNzAyNSwic3ViIjoiYWNjZXNzX3Rva2VuIn0.Ey-w2K_EdH4XfWQgjuHpWMn_RX5k-UwwZRkNVQ7CE18



In [ ]:
client_id = 'ETYHMVN1Q6-100'
secret_key = 'I0CJUFFA03'
redirect_uri = 'https://www.google.com/'
response_type = 'code'
state = 'sample_state'
user_name = 'YJ08526'
APPID_HASH = 'dc1cd1f989e67ad53623c26ac48315e1e88ed60bdd58a2cbc12ea294c248d3db'

In [3]:
session = fyersModel.SessionModel(
    client_id= client_id,
    secret_key= secret_key,
    redirect_uri= redirect_uri,
    response_type= response_type
)

In [4]:
response = session.generate_authcode()

In [5]:
print(response)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=ETYHMVN1Q6-100&redirect_uri=https%3A%2F%2Fwww.google.com%2F&response_type=code&state=None


In [6]:
link = response 

In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time



options = webdriver.ChromeOptions()
# Remove the user-data-dir argument
# options.add_argument("--user-data-dir=/workspaces/Stock_analysis")

driver = webdriver.Chrome(options=options)


# driver = webdriver.Chrome()
driver.get(link)
time.sleep(5)
driver.close()

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir
Stacktrace:
#0 0x55c57e415d0a <unknown>
#1 0x55c57dec65f0 <unknown>
#2 0x55c57df000db <unknown>
#3 0x55c57defbe2f <unknown>
#4 0x55c57df4bf9e <unknown>
#5 0x55c57df4b4c6 <unknown>
#6 0x55c57df3d433 <unknown>
#7 0x55c57df09ea3 <unknown>
#8 0x55c57df0ab01 <unknown>
#9 0x55c57e3dab5b <unknown>
#10 0x55c57e3dea41 <unknown>
#11 0x55c57e3c1c52 <unknown>
#12 0x55c57e3df5b4 <unknown>
#13 0x55c57e3a5f0f <unknown>
#14 0x55c57e403db8 <unknown>
#15 0x55c57e403f96 <unknown>
#16 0x55c57e414b56 <unknown>
#17 0x73695022a609 start_thread


In [21]:
uri = 'https://www.google.com/?s=ok&code=200&auth_code=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhcHBfaWQiOiJFVFlITVZOMVE2IiwidXVpZCI6ImQ2YzNmYzZlMTlhZTRmZWM5ZTYzMjA0OTUyODMyZGRjIiwiaXBBZGRyIjoiIiwibm9uY2UiOiIiLCJzY29wZSI6IiIsImRpc3BsYXlfbmFtZSI6IllKMDg1MjYiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiZmIzZWQxZWE5ZmQwOGIzNzA1NGJlNmM1YmZjYzJiMDg1MmI4Mjg1ZDVmMWNmMGEyYWM0YTdkMyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImF1ZCI6IltcImQ6MVwiLFwiZDoyXCIsXCJ4OjBcIixcIng6MVwiLFwieDoyXCJdIiwiZXhwIjoxNzQ0NTU1OTE5LCJpYXQiOjE3NDQ1MjU5MTksImlzcyI6ImFwaS5sb2dpbi5meWVycy5pbiIsIm5iZiI6MTc0NDUyNTkxOSwic3ViIjoiYXV0aF9jb2RlIn0.eQF-vhV37uEPIm3gfbV5GO-2pRPfJ0EuVdV12gpPe_M&state=None'

In [22]:
auth = uri.split('auth_code=')[1].split('&state')[0]

In [23]:
print(auth)

eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhcHBfaWQiOiJFVFlITVZOMVE2IiwidXVpZCI6ImQ2YzNmYzZlMTlhZTRmZWM5ZTYzMjA0OTUyODMyZGRjIiwiaXBBZGRyIjoiIiwibm9uY2UiOiIiLCJzY29wZSI6IiIsImRpc3BsYXlfbmFtZSI6IllKMDg1MjYiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiZmIzZWQxZWE5ZmQwOGIzNzA1NGJlNmM1YmZjYzJiMDg1MmI4Mjg1ZDVmMWNmMGEyYWM0YTdkMyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImF1ZCI6IltcImQ6MVwiLFwiZDoyXCIsXCJ4OjBcIixcIng6MVwiLFwieDoyXCJdIiwiZXhwIjoxNzQ0NTU1OTE5LCJpYXQiOjE3NDQ1MjU5MTksImlzcyI6ImFwaS5sb2dpbi5meWVycy5pbiIsIm5iZiI6MTc0NDUyNTkxOSwic3ViIjoiYXV0aF9jb2RlIn0.eQF-vhV37uEPIm3gfbV5GO-2pRPfJ0EuVdV12gpPe_M


In [24]:
# Define your Fyers API credentials
client_id = 'ETYHMVN1Q6-100'
secret_key = 'I0CJUFFA03'
redirect_uri = 'https://www.google.com/'

response_type = "code" 
grant_type = "authorization_code"  

# The authorization code received from Fyers after the user grants access
auth_code = auth

# Create a session object to handle the Fyers API authentication and token generation
session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)

# Set the authorization code in the session object
session.set_token(auth_code)

# Generate the access token using the authorization code
response = session.generate_token()

# Print the response, which should contain the access token and other details
print(response)


# ----------------------------------------------------------------------------------
# Sample Success Response               
# ----------------------------------------------------------------------------------          

# {
#   's': 'ok',
#   'code': 200,
#   'message': '',
#   'access_token': 'eyJ0eXAiOi***.eyJpc3MiOiJh***.HrSubihiFKXOpUOj_7***',
#   'refresh_token': 'eyJ0eXAiO***.eyJpc3MiOiJh***.67mXADDLrrleuEH_EE***'
# }


{'access_token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCbi0xcUVySEF2Y1NGbnY5cEhvWkttSGZORF9MdlM0dnNGSHBndmRyR2hHX2pHZi1qTWtYT0RscXhZOGhkSFM3NWF0N1JzWFpLZjNNX0lCbmtqLTJQQ2V0QTk5T1R4QlFaZ05Nby00LWFLZGdoSk1wWT0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiZmIzZWQxZWE5ZmQwOGIzNzA1NGJlNmM1YmZjYzJiMDg1MmI4Mjg1ZDVmMWNmMGEyYWM0YTdkMyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWUowODUyNiIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzQ0NTkwNjAwLCJpYXQiOjE3NDQ1MjU5NTYsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc0NDUyNTk1Niwic3ViIjoiYWNjZXNzX3Rva2VuIn0.JlcjZFdRXkY4T1zR_icp7EXe2k7GxDF31ghOjUzkhaA', 'code': 200, 'message': '', 'refresh_token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCbi0xcUVySEF2Y1NGbnY5cEhvWkttSGZORF9MdlM0dnNGSHBndmRyR2hHX2pHZi1qTWtYT0RscXhZOGhkSFM3NWF0N1JzWFpLZjNNX0lCbmtqLTJQQ2V0QTk5T1R4QlFaZ05Nby00LWFLZGdoSk1wWT0iLCJkaXNw

In [25]:
response

{'access_token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCbi0xcUVySEF2Y1NGbnY5cEhvWkttSGZORF9MdlM0dnNGSHBndmRyR2hHX2pHZi1qTWtYT0RscXhZOGhkSFM3NWF0N1JzWFpLZjNNX0lCbmtqLTJQQ2V0QTk5T1R4QlFaZ05Nby00LWFLZGdoSk1wWT0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiZmIzZWQxZWE5ZmQwOGIzNzA1NGJlNmM1YmZjYzJiMDg1MmI4Mjg1ZDVmMWNmMGEyYWM0YTdkMyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWUowODUyNiIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzQ0NTkwNjAwLCJpYXQiOjE3NDQ1MjU5NTYsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc0NDUyNTk1Niwic3ViIjoiYWNjZXNzX3Rva2VuIn0.JlcjZFdRXkY4T1zR_icp7EXe2k7GxDF31ghOjUzkhaA',
 'code': 200,
 'message': '',
 'refresh_token': 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCbi0xcUVySEF2Y1NGbnY5cEhvWkttSGZORF9MdlM0dnNGSHBndmRyR2hHX2pHZi1qTWtYT0RscXhZOGhkSFM3NWF0N1JzWFpLZjNNX0lCbmtqLTJQQ2V0QTk5T1R4QlFaZ05Nby00LWFLZGdoSk1wWT0iLCJka

In [28]:
access_token = response['access_token']

In [29]:
access_token

'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhdWQiOlsiZDoxIiwiZDoyIiwieDowIiwieDoxIiwieDoyIl0sImF0X2hhc2giOiJnQUFBQUFCbi0xcUVySEF2Y1NGbnY5cEhvWkttSGZORF9MdlM0dnNGSHBndmRyR2hHX2pHZi1qTWtYT0RscXhZOGhkSFM3NWF0N1JzWFpLZjNNX0lCbmtqLTJQQ2V0QTk5T1R4QlFaZ05Nby00LWFLZGdoSk1wWT0iLCJkaXNwbGF5X25hbWUiOiIiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJiZmIzZWQxZWE5ZmQwOGIzNzA1NGJlNmM1YmZjYzJiMDg1MmI4Mjg1ZDVmMWNmMGEyYWM0YTdkMyIsImlzRGRwaUVuYWJsZWQiOiJOIiwiaXNNdGZFbmFibGVkIjoiTiIsImZ5X2lkIjoiWUowODUyNiIsImFwcFR5cGUiOjEwMCwiZXhwIjoxNzQ0NTkwNjAwLCJpYXQiOjE3NDQ1MjU5NTYsImlzcyI6ImFwaS5meWVycy5pbiIsIm5iZiI6MTc0NDUyNTk1Niwic3ViIjoiYWNjZXNzX3Rva2VuIn0.JlcjZFdRXkY4T1zR_icp7EXe2k7GxDF31ghOjUzkhaA'

In [30]:
client_id = client_id
access_token = access_token

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

data = {
    "symbol":"NSE:SBIN-EQ",
    "resolution":"D",
    "date_format":"0",
    "range_from":"1690895316",
    "range_to":"1691068173",
    "cont_flag":"1"
}

response = fyers.history(data=data)
print(response)


{'candles': [[1690848000, 620.2, 620.85, 610.6, 611.85, 13493453], [1690934400, 609.85, 610.5, 594.1, 598.45, 14977497], [1691020800, 598.7, 600.85, 585, 590.5, 27774877]], 'code': 200, 'message': '', 's': 'ok'}


In [1]:
import pandas as pd 
import numpy as np
import datetime
import Analysis
import pandas_ta as ta
import yfinance as yf
import time
from datetime import datetime, time as dt_time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from pretty_html_table import build_table
import pytz
import os
my_variable = os.getenv('AUTO_MAIL')

In [2]:
def index_5min_breakout(ticker,p):
  # yf.download('INFY.NS',interval='1m',period = '1d',ignore_tz = True,multi_level_index=False)
  df = yf.download(ticker, period = p , interval = '5m',ignore_tz = True,multi_level_index=False)
  df = df.reset_index()
  # df['Datetime'] = df['Datetime'].dt.tz_convert('Asia/Kolkata')
  df['symbol'] = ticker.split('.')[0]
  df['month'] = df['Datetime'].dt.month_name()
  df['smma'] = Analysis.calculate_smma(df, period=7)
  df['rsi'] = ta.rsi(df['Close'])
  df['pct'] = df['Close'].pct_change()
  df['dema'] = ta.dema(df['Close'], length=10)
  df['super_trend'] = ta.supertrend(df['High'], df['Low'], df['Close'], length=10, multiplier=3)['SUPERT_10_3.0']
  df['super_trend_color'] = np.where(df['Close'] > df['super_trend'] , "green" , "red")
  df['BBM'] = ta.bbands(close = df['Close'], length=20, std=2)['BBM_20_2.0']
  # df = calculate_fibonacci_pivots(df)
  #identify the position it changes from green to red or viceversa, insert 0 at initial position to avoid blank
  df['changeover'] = np.insert(np.diff(np.where(df['super_trend_color'] == 'green' , 1 , 0)),0,0)
  df['changeover2'] = np.insert(np.diff(np.where(df['super_trend_color'] == 'red' , 1 , 0)),0,0)
  # df['condition'] = np.insert(np.diff(np.where((df['BBM'] < df['smma']) & (df['smma'] < df['dema']) ,1,0)),0,0)
  # df['condition2'] = np.insert(np.diff(np.where((df['BBM'] > df['smma']) & (df['smma'] > df['dema']) ,1,0)),0,0)
  # df['breakout_down'] = np.where((df['changeover'] == 1) & (df['condition'] == 1),"Breakout",np.where((df['changeover2']     == 1) & (df['condition2'] == 1),"Breakdown",np.nan))
  condition1 = np.where((df['BBM'] < df['smma']) & (df['smma'] < df['dema']) ,1,0)
  condition2 = np.where((df['BBM'] > df['smma']) & (df['smma'] > df['dema']) ,1,0)
  df['breakout_down'] = np.where((df['changeover'] == 1) & (condition1),"Breakout",np.where((df['changeover2']== 1) & (condition2),"Breakdown",np.nan))
  
  # np.where(df['changeover'] == 1 & df['condition'] == 1)
  # df = df.iloc[np.where(((df['changeover'] == 1) & (df['condition'] == 1)) | ((df['changeover2'] == 1) & (df['condition2'] == 1)))]
  df = df.iloc[np.where(((df['changeover'] == 1) & (condition1)) | ((df['changeover2'] == 1) & (condition2)))]
  
  # monthly = yf.download(ticker , period = '3mo' , interval = '1d')
  # monthly = monthly.droplevel(1,axis=1).reset_index()
  # monthly = calculate_fibonacci_pivots(monthly)
  # df = df.merge(monthly[['Pivot','R1','R2','R3','S1','S2','S3']], left_on = [df['Datetime'].dt.month , df['Datetime'].dt.year] , right_on = [monthly['Date'].dt.month , monthly['Date'].dt.year] )
  # df.drop(columns = ['key_1','key_0'], axis = 1 ,inplace = True )
  return df

In [7]:
def send_mail(output):
  sender_email = 'ragothaman4010@gmail.com'
  # receiver_email = ['ragothaman4010@gmail.com','narendran2cool@gmail.com ','esyuvaraj@gmail.com','sriganishka@gmail.com']
  receiver_email = ['ragothaman4010@gmail.com']
  subject = 'Today option calls'
  body = f"""
  <html>
    <body>
      <p><h3>Hi,</h3><br>
        Please find the Today's Option calls below:<br>
      </p>
      {output}
        <br>
      <h3>
      Regards,<br>
      LIGTHS Team
      </h3>
    </body>
  </html>
  """
  if not my_variable:
        print("AUTO_MAIL environment variable is not set. Email not sent.")
        return
  # Create MIME message
  msg = MIMEMultipart('alternative')
  msg['Subject'] = subject
  msg['From'] = sender_email
  msg['To'] = ', '.join(receiver_email)

  # Attach the HTML body
  msg.attach(MIMEText(body, 'html'))

  # # Attach the plot.html file
  # filename = "combined_plot.html"
  # with open(filename, "rb") as attachment:
  #     part = MIMEBase("application", "octet-stream")
  #     part.set_payload(attachment.read())

  # # Encode file in ASCII characters to send by email    
  # encoders.encode_base64(part)

  # # Add header as key/value pair to attachment part
  # part.add_header(
  #     "Content-Disposition",
  #     f"attachment; filename= {filename}",
  # )

  # # Attach the file to the message
  # msg.attach(part)

  # Send the email
  with smtplib.SMTP('smtp.gmail.com', 587) as server:
      server.starttls()
      server.login(sender_email, my_variable)
      server.sendmail(sender_email, receiver_email, msg.as_string())

  print("Email sent successfully!")



In [4]:
IST = pytz.timezone('Asia/Kolkata')

In [6]:
def is_market_hours():
    now_ist = datetime.now(IST).time()
    return dt_time(9, 15) <= now_ist <= dt_time(20, 00)

In [98]:

now_ist = datetime.now(IST).time()
if is_market_hours():
    try:
        list_ = {'^NSEI':"Nifty 50",'^NSEBANK':"BankNifty",'NIFTY_FIN_SERVICE.NS':"FinNifty"}
        dff= []
        # chk=[]
        for i in list_:
            df = index_5min_breakout(i,'1d')[['Datetime','Close','breakout_down']]
            df['index'] = list_[i]
            dff.append(df)
        breakout = pd.concat(dff)
        breakout= breakout.sort_values(by=['Datetime'])
        chk.extend(breakout['Datetime'])
        breakout = breakout[~breakout['Datetime'].isin(chk)]
        if len(breakout) == 0:
            output = "<h1>No Breakout or Breakdown till now</h1>"  # HTML for the message
        else:
            output = build_table(breakout, 'yellow_dark', font_size='small', font_family='Open Sans, sans-serif',
                                text_align='center', width='Auto', index=False, even_color='black', even_bg_color='white')

        if len(breakout) > 0:
            send_mail(output)
        else: 
            pass
    except Exception as e:
        print(f"Error occurred: {e}")



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [82]:

# Define the data
data = [
{"Datetime": "2025-05-13 12:50:00", "Close": 24614.949219, "breakout_down": "Breakdown", "index": "Nifty 50"},
{"Datetime": "2025-05-13 13:15:00", "Close": 54927.300781, "breakout_down": "Breakdown", "index": "BankNifty"},
{"Datetime": "2025-05-13 13:15:00", "Close": 26211.199219, "breakout_down": "Breakdown", "index": "FinNifty"}
]

# Create the DataFrame
df = pd.DataFrame(data)


In [39]:
dff=[]
for i in list_:
    df = index_5min_breakout(i,'1d')[['Datetime','Close','breakout_down']]
    df['index'] = list_[i]
    dff.append(df)
breakout = pd.concat(dff)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [100]:
breakout[~breakout['Datetime'].isin(chk)]

,Datetime,Close,breakout_down,index


In [78]:
chk.extend(df['Datetime'].values.tolist())

In [99]:
chk

[Timestamp('2025-05-15 13:05:00'), Timestamp('2025-05-15 13:05:00')]